In [1]:
import hashlib
import random
from sage.crypto.util import ascii_to_bin
from sage.crypto.util import bin_to_ascii
from sage.misc.prandom import randrange
from sage.crypto.util import ascii_integer

In [2]:
def HTR(ID,p,hashfcn):
    hashfcn.update(str(ID))
    h = int(hashfcn.hexdigest(),16)
    intTomod = mod(h,p)
    return intTomod

def HashToPoint(E,p,q,ID,hashfcn):
    y = HTR(ID,p,hashfcn)
    x = (y^2 - 1)^((2*p-1)/3) % p
    Q2 = E(x,y)
    Q = (int((p + 1)/q))*Q2
    return Q

def rand_point(E,r):
    P = E.random_point()
    P2 = (12*r)*P
    if P2 == 0: return -1
    return P2

def custom_xor(a,b,l):
    num = []
    for i in range(0,l): 
        num.append((int(a[i]) + int(b[i])) % 2)
    return "".join(map(str,num))

In [3]:
def setup_basic(sec_n):
    if sec_n == 1024: n_p = 512; n_q = 160; hashfcn = hashlib.sha1()
    if sec_n == 2048: n_p = 1024; n_q = 224; hashfcn = hashlib.sha224()
    if sec_n == 3072: n_p = 1536; n_q = 256; hashfcn = hashlib.sha256()
    if sec_n == 7680: n_p = 3840; n_q = 284; hashfcn = hashlib.sha384()
    if sec_n == 15360: n_p = 7680; n_q = 512; hashfcn = hashlib.sha512()

    aux = -1
    
    while(aux == -1):
    
        q = random_prime(pow(2,n_q)-1,False,pow(2,n_q - 1))
        
        print "bits q = ",q.nbits()
        
        l = n_p - n_q - 4 + 1
    
        r = random.randint(2^(l-1),2^l)
        p = 12*r*q - 1
    
        while not p.is_prime():
            r = random.randint(2^(l-1),2^l)
            p = 12*r*q - 1
            
        print "bits p ",p.nbits()

        E = EllipticCurve(GF(p),[0,1]);
        
        aux = rand_point(E,q)
        
    P = aux
    
    s = random.randint(2,q-1)
    Ppub = s*P
    
    params = E,p,q,P,Ppub,hashfcn,s
    
    return params

def extract_basic(ID, E,p,q,hashfcn, s):
    Qid = HashToPoint(E,p,q,ID,hashfcn)
    return s*Qid

def encryption_basic(ID,E,p,q,P,Ppub,hashfcn,M):
    
    if len(M) % 8 > 0: pad = 8 - len(M) % 8; M = M + (' ' * pad) 
        
    bin = BinaryStrings()
    M = bin.encoding(M)
    
    M = Integer(str(M),base=2).binary()
        
    r = random.randint(1,q-1)
    
    Qid = HashToPoint(E,p,q,ID,hashfcn)
    
    Fp2 = GF(p^2,'a')
    omega = Fp2.zeta(3)

    Ep2 = EllipticCurve(Fp2,[0,1])
    
    P2 = Ep2((Qid[0],Qid[1]))
    
    betaQ = Ep2(omega*Qid[0],Qid[1])
    
    gID = P2.weil_pairing(betaQ,p+1)
    
    gIDr = pow(gID,r)
    
    C1 = r*P
    
    H2 =  ZZ(HTR(gIDr,q,hashfcn)).binary()
    
    C2 = custom_xor(M,H2,len(M))
    
    return C1,C2

In [7]:
n = 1024

E,p,q,P,Ppub,hashfcn,s = setup_basic(n)
Sid = extract_basic("sergi",E,p,q,hashfcn,s)
C1,C2 = encryption_basic("sergi",E,p,q,P,Ppub,hashfcn,"sergi")  
print C1,C2

bits q =  160
bits p  512
(6992276133570884251429096472801222715653885360772076189674453188403753574096855628553841998597275540810511195458776829407775923387674944828706786168440937 : 7108272669308311428805871328340208593942690725835014017808309038258469313449382143445852631626915662525456196835939953880994876014556647157813613105199540 : 1) 000000000010110110001100111100111011011001011010101101010011101


In [8]:
def decrypt_basic(E,p,q,P,Ppub,hashfcn,C1,C2,Sid):
    
    Fp2 = GF(p^2,'a')
    omega = Fp2.zeta(3)

    Ep2 = EllipticCurve(Fp2,[0,1])
    
    P2 = Ep2((Sid[0],Sid[1]))
    
    betaC1 = Ep2(omega*C1[0],C1[1])
    
    fID = P2.weil_pairing(betaC1,q)
    
    H2 = ZZ(HTR(fid,q,hashfcn)).binary()
    
    print len(C2)
    print len(H2)

decrypt_basic(E,p,q,P,Ppub,hashfcn,C1,C2,Sid)

ValueError: points must both be n-torsion